In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os

In [2]:
class BasicBlock(nn.Module):
  def __init__(self, in_planes, planes, stride=1):
    super(BasicBlock, self).__init__()

    # Use 3x3 filter  
    self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)

    # Use 3x3 filter
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)

    self.shortcut = nn.Sequential() # identity
    if stride != 1: # Not identity mapping
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
          nn.BatchNorm2d(planes)
      )

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    out += self.shortcut(x) # skip connection
    out = F.relu(out)
    return out

In [3]:
class ResNet(nn.Module):
  def __init__(self, block, num_blocks, num_classes=10):
    super(ResNet, self).__init__()
    self.in_planes = 64

    # Use 64 3x3 filters
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
    self.linear = nn.Linear(512, num_classes)

  def _make_layer(self, block, planes, num_blocks, stride):
    strides = [stride] + [1] * (num_blocks - 1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_planes, planes, stride))
      self.in_planes = planes
    return nn.Sequential(*layers)

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = F.avg_pool2d(out, 4)
    out = out.view(out.size(0), -1)
    out = self.linear(out)
    return out

In [4]:
def ResNet18():
  return ResNet(BasicBlock, [2, 2, 2, 2])

In [5]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
                                       transforms.RandomCrop(32, padding=4),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
])

transform_test = transforms.Compose([
                                      transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='/data', train=False, transform=transform_test, download=True)

train_laoder = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)

In [7]:
def train(epoch):
  print("######################")
  print("Train epoch: ", epoch)
  net.train()
  train_loss = 0
  correct = 0
  total = 0

  for batch_idx, (inputs, targets) in enumerate(train_laoder):
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()

    outputs = net(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    
    optimizer.step()
    train_loss += loss.item()
    _, predicted = outputs.max(1)

    total += targets.size(0)
    correct += predicted.eq(targets).sum().item()

    if batch_idx % 100 == 0:
      print("\nCurrent batch: ", str(batch_idx))
      print("Current train accuracy: ", str(predicted.eq(targets).sum().item() / targets.size(0)))
      print("Current train loss: ", loss.item())
      
  print("\nTotal train accuracy: ", 100 * correct / total)
  print("Total train loss: ", train_loss)

In [11]:
def test(epoch):
  print("######################")
  print("Test epoch: ", epoch)
  net.eval()
  loss = 0
  correct = 0
  total = 0

  for batch_idx, (inputs, targets) in enumerate(test_loader):
    inputs, targets = inputs.to(device), targets.to(device)
    total += targets.size(0)

    outputs = net(inputs)
    loss += criterion(outputs, targets).item()

    _, predicted = outputs.max(1)
    correct += predicted.eq(targets).sum().item()
  
  print("\nTest accuracy: ", 100. * correct / total)
  print("Test average loss: ", loss / total)

  state = {
      'net': net.state_dict()
  }

  if not os.path.isdir('checkpoint'):
    os.mkdir('checkpoint')
  torch.save(state, './checkpoint/' + file_name)
  print("Model saved")

In [9]:
def adjust_learning_rate(optimizer, epoch):
  lr = learning_rate
  if epoch >= 100:
    lr /= 10
  if epoch >= 150:
    lr /= 10
  for param_group in optimizer.param_groups:
    param_group['lr'] = lr
  
  print("Current learning_rate: ", lr)

In [12]:
for epoch in range(20):
  adjust_learning_rate(optimizer, epoch)
  train(epoch)
  test(epoch)

Current learning_rate:  0.1
######################
Train epoch:  0

Current batch:  0
Current train accuracy:  0.3984375
Current train loss:  1.61381196975708

Current batch:  100
Current train accuracy:  0.4921875
Current train loss:  1.5531542301177979

Current batch:  200
Current train accuracy:  0.4453125
Current train loss:  1.3701614141464233

Current batch:  300
Current train accuracy:  0.515625
Current train loss:  1.4072171449661255

Total train accuracy:  48.352
Total train loss:  549.9188067913055
######################
Test epoch:  0

Test accuracy:  32.44
Test average loss:  0.0225202271938324
Model saved
Current learning_rate:  0.1
######################
Train epoch:  1

Current batch:  0
Current train accuracy:  0.46875
Current train loss:  1.4805231094360352

Current batch:  100
Current train accuracy:  0.5703125
Current train loss:  1.1948707103729248

Current batch:  200
Current train accuracy:  0.6484375
Current train loss:  1.1820855140686035

Current batch:  300
Cu